# NerdWallet - Web scraping

In [39]:
%pip install -r requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [197]:
import requests, time
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import pandas as pd

In [257]:
base_url = 'https://www.nerdwallet.com/the-best-credit-cards'

In [326]:
def get_cards_info(response, t):
    page = BeautifulSoup(response.text, 'html.parser')
    cards = page.findAll('a', {'class': 'MuiTypography-root MuiTypography-headline MuiLink-root MuiLink-underlineHover css-7wcx0j'})
    cards_info = page.findAll('div', {'class': 'MuiBox-root css-q5fqw0'})
    cards_meta_ls = []
    card_meta = {}
    rank = 1
    for card in cards:
        card_meta['card name'] = card.text
        card_meta['ref link'] = card['href']
        card_meta['category'] = t
        card_meta['rank in category'] = rank
        rank += 1
        cards_meta_ls.append(card_meta.copy())
    cards_ls = []
    card_info = {}
    for c in cards_info:
        info_key = c.findAll('p', {'class': 'MuiTypography-root MuiTypography-body1 css-19gg6ql'})
        info_value = c.findAll('div', {'class': 'MuiBox-root css-osq69c'})
        for key, value in zip(info_key, info_value):
            if value.find('span', {'class': 'MuiBox-root css-1baulvz'}):
                detail = value.find('span', {'class': 'MuiBox-root css-1baulvz'})
                card_info[key.text] = value.text + "(" + detail.span["aria-label"] + ")"
            else:
                card_info[key.text] = value.text
        cards_ls.append(card_info.copy())
    cards_meta_pd = pd.DataFrame(cards_meta_ls)
    cards_pd = pd.DataFrame(cards_ls)
    result_pd = pd.concat([cards_meta_pd, cards_pd], axis=1)
    return result_pd

In [327]:
tabs = ["monthly-best", "travel", "balance-transfer", "low-interest", "cash-back", "rewards", "building-credit", "student", "fair-credit", "business"]
result = pd.DataFrame({})
for t in tabs:
    url = f"{base_url}/{t}"
    try:
        response = requests.get(url)
        response.raise_for_status()
    except HTTPError as hp:
        print(hp)
    result = pd.concat([result, get_cards_info(response, t)], ignore_index=True)

In [330]:
result

,card name,ref link,category,rank in category,Annual fee,Rewards rate,Intro offer,Recommended Credit Score,Intro APR,Regular APR,Balance transfer fee,Minimum deposit
0,Wells Fargo Active Cash® Card,https://www.nerdwallet.com/redirect/b954a46c-c...,monthly-best,1,$0,2%cashback(Earn unlimited 2% cash rewards on p...,$200(Earn a $200 cash rewards bonus after spen...,,NaN,NaN,NaN,NaN
1,Chase Freedom Unlimited®,https://www.nerdwallet.com/redirect/84e1494c-b...,monthly-best,2,$0,1.5%-6.5%cashback(Enjoy 6.5% cash back on trav...,$300(Earn an additional 1.5% cash back on ever...,,NaN,NaN,NaN,NaN
2,Capital One Venture Rewards Credit Card,https://www.nerdwallet.com/redirect/1fc99a80-1...,monthly-best,3,$95,2x-5xmiles(Earn unlimited 2X miles on every pu...,"75,000miles(Enjoy a one-time bonus of 75,000 m...",,NaN,NaN,NaN,NaN
3,Chase Sapphire Preferred® Card,https://www.nerdwallet.com/redirect/7dce19da-5...,monthly-best,4,$95,1x-5xpoints(Enjoy benefits such as a $50 annua...,"60,000points(Earn 60,000 bonus points after yo...",,NaN,NaN,NaN,NaN
4,Capital One SavorOne Cash Rewards Credit Card,https://www.nerdwallet.com/redirect/cfeca922-5...,monthly-best,5,$0,1%-10%cashback(Earn unlimited 3% cash back on ...,$200(Earn a one-time $200 cash bonus after you...,,NaN,NaN,NaN,NaN
5,Discover it® Cash Back,https://www.nerdwallet.com/redirect/a4a36a73-0...,monthly-best,6,$0,1%-5%cashback(Earn 5% cash back on everyday pu...,Cashback Match™(INTRO OFFER: Unlimited Cashbac...,,NaN,NaN,NaN,NaN
6,Discover it® Balance Transfer,https://www.nerdwallet.com/redirect/27f6639e-1...,monthly-best,7,$0,1%-5%cashback(Earn 5% cash back on everyday pu...,Cashback Match™(INTRO OFFER: Unlimited Cashbac...,,NaN,NaN,NaN,NaN
7,Blue Cash Preferred® Card from American Express,https://www.nerdwallet.com/redirect/2681330d-4...,monthly-best,8,"$0 intro annual fee for the first year, then $95.",1%-6%cashback(6% Cash Back at U.S. supermarket...,$250(Earn a $250 statement credit after you sp...,,NaN,NaN,NaN,NaN
8,Chase Sapphire Preferred® Card,https://www.nerdwallet.com/redirect/7dce19da-5...,travel,1,$95,1x-5xpoints(Enjoy benefits such as a $50 annua...,"60,000points(Earn 60,000 bonus points after yo...",,NaN,NaN,NaN,NaN
9,Capital One Venture Rewards Credit Card,https://www.nerdwallet.com/redirect/1fc99a80-1...,travel,2,$95,2x-5xmiles(Earn unlimited 2X miles on every pu...,"75,000miles(Enjoy a one-time bonus of 75,000 m...",,NaN,NaN,NaN,NaN


In [331]:
result.to_json("./NerdWallet_creditcard.json")